In [3]:
import pandas as pd
import requests
import time
import datetime
from io import StringIO
from bs4 import BeautifulSoup

# 展開報表結果
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.width', 1000)
# 偽瀏覽器
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

# 取得所有股市代碼
def getAllCode():
    # 上市代碼網址
    url = 'https://isin.twse.com.tw/isin/C_public.jsp?strMode=2'
    # 下載網站內容，並用pandas轉換成 dataframe
    r = requests.get(url, headers=headers)
    r.encoding = 'big5'

    dfs = pd.read_html(StringIO(r.text), encoding='big-5')
    df_stock = pd.concat([df for df in dfs if df.shape[1] == 7])
    df_stock = df_stock[df_stock[5] == 'ESVUFR']
    df_stock.reset_index(inplace=True)
    df_stock = df_stock[0].str[:4]

    # 上櫃代碼網址
    url = 'https://isin.twse.com.tw/isin/C_public.jsp?strMode=4'
    r = requests.get(url, headers=headers)
    r.encoding = 'big5'

    dfs = pd.read_html(StringIO(r.text), encoding='big-5')
    df_otc = pd.concat([df for df in dfs if df.shape[1] == 7])
    df_otc = df_otc[df_otc[5] == 'ESVUFR']
    df_otc.reset_index(inplace=True)
    df_otc = df_otc[0].str[:4]

    df_all_code = pd.concat([df_stock,df_otc])
    df_all_code = df_all_code.reset_index(drop=True) # 1719筆資料
    return df_all_code

# 取得前一天收盤價
def getAllCodeWithPrice(df_all_code):
    # 上市代碼網址
    url = 'http://www.twse.com.tw/exchangeReport/STOCK_DAY_ALL?response=open_data'


    # 下載網站內容，並用pandas轉換成 dataframe
    r = requests.get(url, headers=headers)
    r.encoding = 'UTF-8'

    df=pd.read_csv(StringIO(r.content.decode('utf-8')))
    df = df[df['收盤價'] != '----' ]
    df_stock = df[['證券代號','收盤價']]

    # 上櫃代碼網址
    url = 'http://www.tpex.org.tw/web/stock/aftertrading/otc_quotes_no1430/stk_wn1430_result.php?l=zh-tw&se=EW&o=data'

    r = requests.get(url, headers=headers)
    r.encoding = 'UTF-8'

    df=pd.read_csv(StringIO(r.content.decode('utf-8')))
    df = df.rename(columns={'代號': '證券代號', '收盤': '收盤價'})
    df = df[df['收盤價'] != '----' ]
    df_otc = df[['證券代號','收盤價']]
    df_all_code_with_closing_price = pd.concat([df_stock,df_otc])
    df_all_code_with_closing_price['收盤價'] = df_all_code_with_closing_price['收盤價'].str.replace(',','')
    df_all_code_with_closing_price = df_all_code_with_closing_price.reset_index(drop=True)

    # 過濾出只包含上市上櫃資料
    df_all_code_with_closing_price = df_all_code_with_closing_price[df_all_code_with_closing_price['證券代號'].isin(df_all_code)]
    df_all_code_with_closing_price = df_all_code_with_closing_price.reset_index(drop=True)
    return  df_all_code_with_closing_price #1703筆資料
    

df_all_code = getAllCode()
df_all_code_with_price = getAllCodeWithPrice(df_all_code)
df_all_code_with_price






,證券代號,收盤價
0,1101,40.30
1,1102,40.55
2,1103,15.75
3,1104,20.05
4,1108,11.90
5,1109,19.10
6,1110,16.65
7,1201,19.80
8,1203,33.60
9,1210,44.75


In [4]:
# 取得標的清單
# @param year_trace 追蹤年數
# @param dividend_yield 殖利率門檻
# @param stock_list 股票代碼和收盤價
def getTargetList(year_trace,dividend_yield,stock_list):
    print('處理資料筆數：' + str(stock_list.shape[0]))
    df = stock_list
    result = []
    for index, stock in df.iterrows():
        time.sleep(0.8)
        print('已經處理筆數：' + str(index+1) + ',代碼：' + stock['證券代號'])
        #print(stock['證券代號'] + ',' + stock['收盤價'])
        if float(stock['收盤價']) == 0:
            continue
        url = 'https://tw.stock.yahoo.com/d/s/dividend_' + stock['證券代號'] + '.html'
        r = requests.get(url, headers=headers)
        r.encoding = 'big5'
        dfs = pd.read_html(StringIO(r.text), encoding='big5')
        df = pd.concat([df for df in dfs if df.shape[1] == 7])
        df[0] = df[0].str[:3]
        now = datetime.datetime.now()
        # 欄位字串轉數字
        df[0] = pd.to_numeric(df[0], 'coerce')
        df[6] = pd.to_numeric(df[6], 'coerce')
        df = df[df[0] != now.year-1911]
        df = df[~df[0].isnull()]
        df = df[df[1] != '-']
        df = df[[0,6]]
        df = df.rename(columns={0: '年度', 6: '股利'}).groupby(['年度']).sum().sort_index(ascending=False).reset_index()
        
        i=1
        year_temp=0
        success = False
        for index, row in df.iterrows():
            now = datetime.datetime.now()
            # 確認第一筆資料為去年
            if year_temp == 0 and row['年度'] != now.year - 1912:
                success = False
                break
            
            # 檢查追蹤年數內是否都有發放股利
            if year_temp != 0 and year_temp != row['年度']+1:
                success = False
                break
            # 檢查是否到達殖利率門檻
            if float(row['股利']) / float(stock['收盤價']) * 100 < dividend_yield:
                success = False
                break
            
            # 到達追蹤年數
            if i >= year_trace:
                success = True
                break

            i += 1
            year_temp = row['年度']
        
        if success:
            result.append(stock['證券代號'])
    return result

target_list = getTargetList(5,7,df_all_code_with_price)
df = df_all_code_with_price[df_all_code_with_price['證券代號'].isin(target_list)]
df

處理資料筆數：1697
已經處理筆數：1,代碼：1101
已經處理筆數：2,代碼：1102
已經處理筆數：3,代碼：1103
已經處理筆數：4,代碼：1104
已經處理筆數：5,代碼：1108
已經處理筆數：6,代碼：1109
已經處理筆數：7,代碼：1110
已經處理筆數：8,代碼：1201
已經處理筆數：9,代碼：1203
已經處理筆數：10,代碼：1210
已經處理筆數：11,代碼：1213
已經處理筆數：12,代碼：1215
已經處理筆數：13,代碼：1216
已經處理筆數：14,代碼：1217
已經處理筆數：15,代碼：1218
已經處理筆數：16,代碼：1219
已經處理筆數：17,代碼：1220
已經處理筆數：18,代碼：1225
已經處理筆數：19,代碼：1227
已經處理筆數：20,代碼：1229
已經處理筆數：21,代碼：1231
已經處理筆數：22,代碼：1232
已經處理筆數：23,代碼：1233
已經處理筆數：24,代碼：1234
已經處理筆數：25,代碼：1235
已經處理筆數：26,代碼：1236
已經處理筆數：27,代碼：1256
已經處理筆數：28,代碼：1301
已經處理筆數：29,代碼：1303
已經處理筆數：30,代碼：1304
已經處理筆數：31,代碼：1305
已經處理筆數：32,代碼：1307
已經處理筆數：33,代碼：1308
已經處理筆數：34,代碼：1309
已經處理筆數：35,代碼：1310
已經處理筆數：36,代碼：1312
已經處理筆數：37,代碼：1313
已經處理筆數：38,代碼：1314
已經處理筆數：39,代碼：1316
已經處理筆數：40,代碼：1319
已經處理筆數：41,代碼：1321
已經處理筆數：42,代碼：1323
已經處理筆數：43,代碼：1324
已經處理筆數：44,代碼：1325
已經處理筆數：45,代碼：1326
已經處理筆數：46,代碼：1337
已經處理筆數：47,代碼：1338
已經處理筆數：48,代碼：1339
已經處理筆數：49,代碼：1340
已經處理筆數：50,代碼：1341
已經處理筆數：51,代碼：1402
已經處理筆數：52,代碼：1409
已經處理筆數：53,代碼：1410
已經處理筆數：54,代碼：1413
已經處理筆數：55,代碼：1414
已經處理筆數：

,證券代號,收盤價
77,1452,15.45
86,1464,17.85
354,2451,62.70
413,2542,42.90
660,4557,62.50
746,6115,40.95
753,6136,17.25
777,6206,61.30
782,6216,25.50
854,8070,20.10
